In [1]:
using Pkg

In [6]:
Pkg.add("StatsBase")
Pkg.add("ClassImbalance")
Pkg.add("ROCAnalysis")
using Pkg
using DataFrames
using CSV
using Plots
using GLM
using StatsBase
using Lathe
using MLBase
using ClassImbalance
using ROCAnalysis

# Enable printing of 1000 columns:
ENV["COLUMNS"] = 1000

  Resolving package versions...
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\aniru\.julia\environments\v1.5\Manifest.toml`
┌ Info: Precompiling ROCAnalysis [f535d66d-59bb-5153-8d2b-ef0a426c6aff]
└ @ Base loading.jl:1278


1000

In [7]:
# To Read File
df = DataFrame(CSV.File("Churn_Modelling.csv"))
first(df,5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0


In [11]:
# Summary Of Data
println(size(df))
describe(df)

(10000, 14)


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,RowNumber,5000.5,1,5000.5,10000,,,Int64
2,CustomerId,1.56909e7,15565701,1.56907e7,15815690,,,Int64
3,Surname,,Abazu,,Zuyeva,2932,,String
4,CreditScore,650.529,350,652.0,850,,,Int64
5,Geography,,France,,Spain,3,,String
6,Gender,,Female,,Male,2,,String
7,Age,38.9218,18,37.0,92,,,Int64
8,Tenure,5.0128,0,5.0,10,,,Int64
9,Balance,76485.9,0.0,97198.5,2.50898e5,,,Float64


In [12]:
# Checking The Name Of The Columns In Data.
names(df)

14-element Array{Symbol,1}:
 :RowNumber
 :CustomerId
 :Surname
 :CreditScore
 :Geography
 :Gender
 :Age
 :Tenure
 :Balance
 :NumOfProducts
 :HasCrCard
 :IsActiveMember
 :EstimatedSalary
 :Exited

In [13]:
# Checking The Number Of Responses In The Response Variable 
countmap(df.Exited)

Dict{Int64,Int64} with 2 entries:
  0 => 7963
  1 => 2037

# One Hot Encoding

In [16]:
Lathe.preprocess.OneHotEncode(df,:Geography)
Lathe.preprocess.OneHotEncode(df,:Gender)
select!(df, Not([:RowNumber, :CustomerId, :Surname, :Geography, :Gender, :Male]))

first(df,5)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany,Female
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Bool,Bool,Bool,Bool
1,619,42,2,0.0,1,1,1,1.01349e5,1,1,0,0,1
2,608,41,1,83807.9,1,0,1,1.12543e5,0,0,1,0,1
3,502,42,8,1.59661e5,3,1,0,1.13932e5,1,1,0,0,1
4,699,39,1,0.0,2,0,0,93826.6,0,1,0,0,1
5,850,43,2,1.25511e5,1,1,1,79084.1,0,0,1,0,1


# TRAIN TEST SPLIT

In [18]:
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.75);

# MODEL BUILDING

In [20]:
fm = @formula(Exited ~ CreditScore + Age + Tenure +Balance +NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France +Spain)
logit = glm(fm, train, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},ProbitLink},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Exited ~ 1 + CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain

Coefficients:
──────────────────────────────────────────────────────────────────────────────────────
                        Coef.  Std. Error       z  Pr(>|z|)     Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.95766      0.166273    -11.77    <1e-31  -2.28355      -1.63177
CreditScore      -0.000307902  0.00018358   -1.68    0.0935  -0.000667712   5.19076e-5
Age               0.0416341    0.00167323   24.88    <1e-99   0.0383546     0.0449136
Tenure           -0.00665231   0.00615402   -1.08    0.2797  -0.018714      0.00540933
Balance           1.6508e-6   

# MODEL PREDICTION AND EVALUATION

In [35]:
#  Predict The Targer Variable On Test Data
prediction = predict(logit, test)

2530-element Array{Union{Missing, Float64},1}:
 0.167089058333461
 0.2588221197456982
 0.09994399226210413
 0.13177448414676038
 0.023959875880486835
 0.01606921213838634
 0.20053346441560138
 0.11650207660489639
 0.05772575577698183
 0.5305438208262888
 0.43941186653593983
 0.24817123563811003
 0.3908707029677382
 ⋮
 0.04233966811226046
 0.056063508010365484
 0.046084815510856046
 0.4433909439263192
 0.2605319637980514
 0.1634605596393764
 0.18462106475104717
 0.36376215009725077
 0.2214321784047832
 0.18179750676955142
 0.040817854368545146
 0.15135444104009682